In [2]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util.sys import get_path, shift_skeleton
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from sparse_util import dilate, zhangSuen
from realign import realign, reconnect, realign2
from util.sys import get_path
import pandas as pd
from datetime import datetime,timedelta
import ast
import os
import scipy.sparse
import scipy.io as sio

import pickle

In [3]:
plate = 3
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']

In [2]:
plate = 9
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[4:8]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
skels_aligned = []
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    dilated = read_mat(path_snap+'/Analysis/dilated.mat')['dilated']
    skel = read_mat(path_snap+'/Analysis/skeleton_realigned.mat')['skeleton']
    skels_aligned.append(scipy.sparse.dok_matrix(skel))

In [3]:
nx_graph_poss=[generate_nx_graph(from_sparse_to_graph(skeleton)) for skeleton in skels_aligned]
nx_graphs_aligned=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[clean_degree_4(prune_graph(nx_graph),poss_aligned[i])[0] for i,nx_graph in enumerate(nx_graphs_aligned)]

[1465.453887]
cleaning, number of nodes before 4217
number of unsolved cases 208
number of unsolved cases 110
number of unsolved cases 102
number of unsolved cases 102
end cleaning, number of nodes after 3493
[1564.79882]
cleaning, number of nodes before 4292
number of unsolved cases 173
number of unsolved cases 90
number of unsolved cases 83
number of unsolved cases 83
number of unsolved cases 83
number of unsolved cases 83
end cleaning, number of nodes after 3613
[2003.15984]
cleaning, number of nodes before 4840
number of unsolved cases 144
number of unsolved cases 59
number of unsolved cases 54
number of unsolved cases 54
end cleaning, number of nodes after 4006
[2568.74304]
cleaning, number of nodes before 5520
number of unsolved cases 276
number of unsolved cases 114
number of unsolved cases 98
number of unsolved cases 89
number of unsolved cases 87
number of unsolved cases 83
number of unsolved cases 83
end cleaning, number of nodes after 4535


In [ ]:
for i,g in enumerate(nx_graph_pruned):
    directory_name=f'2020{dates[i]}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    path_save = path_snap +'/Analysis/nx_graph_pruned.p'
    pos = poss_aligned[i]
    pickle.dump((g,pos),open( path_save, "wb" ))

In [8]:
downstream_graphs = []
downstream_pos = []
begin=len(dates)-1
downstream_graphs=[nx_graph_pruned[begin]]
downstream_poss=[poss_aligned[begin]]
for i in range (begin-1,-1,-1):
    print("i=",i)
    new_graphs,new_poss = second_identification(nx_graph_pruned[i],downstream_graphs[0],poss_aligned[i],downstream_poss[0],50,downstream_graphs[1:],downstream_poss[1:],tolerance=30)
    downstream_graphs=new_graphs
    downstream_poss=new_poss

i= 2


/home/mrozemul/anaconda3/envs/extract_skel/lib/python3.7/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


first_id 10.259893417358398
relabel 0.04011249542236328
tip_id 813.6350462436676
2970 0 [21844 38183]
2970 1 [21835 38172]
i= 1
first_id 10.730165481567383
relabel 0.07279706001281738


/home/mrozemul/anaconda3/envs/extract_skel/lib/python3.7/site-packages/ipykernel_launcher.py:332: RuntimeWarning: invalid value encountered in arccos


tip_id 685.9996612071991
2730 0 [22073 38487]
2730 1 [22077 38480]
2730 2 [22075 38477]
i= 0
first_id 9.933119297027588
relabel 0.10545039176940918
tip_id 644.8557379245758
2700 0 [22195 38616]
2700 1 [22190 38624]
2700 2 [22198 38618]
2700 3 [22195 38617]


In [9]:
nx_graph_pruned=downstream_graphs
poss_aligned=downstream_poss

In [13]:
for i,g in enumerate(nx_graph_pruned):
    directory_name=f'2020{dates[i]}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    path_save = path_snap +'/Analysis/nx_graph_pruned.p'
    pos = poss_aligned[i]
    pickle.dump((g,pos),open( path_save, "wb" ))

In [18]:
for i, date in enumerate(dates):
    tab = from_nx_to_tab(nx_graph_pruned[i],poss_aligned[i])
    directory_name=f'2020{dates[i]}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    path_save = path_snap +'/Analysis/graph_full_labeled.mat'
    sio.savemat(path_save, {name: col.values for name, col in tab.items()})
